This paper: 
https://www.z-riskengine.com/media/hqtnwlmb/a-one-parameter-representation-of-credit-risk-and-transition-matrices.pdf [1]
describes an approach to stress transition matrices using a single parameter based on the standard normal distribution. 
The method can be described in the following steps: 

1. Identify the base transition matrix.
2. Set cummulative probabilities by starting from default and working your way towards AAA; for example: 
the row : 90%  5% 3% 1% 0.7% 0.3% becomes: 100% 10% 5% 2% 1% 0.3% 
3. Calculate the normal cummulative probability for each; this is your Z=0 position.
4. Consider a value of Z (say Z=1); apply the following factor to the cummulative probabilities of step 3: sqrt(rho) * Z
5. Reverse the normal probabilities back to cummulative numbers between 0% and 100%
6. Unravel the cummulation. 
(if you've done it right, for Z=0 you should get back to your original transition matrix).

The paper is quite old, from 1998, i.e. it has missed the credit crunch crisis and the data seem not to capture the 1932 depression period. 
nevertheless the methodology could be expanded and applied to more recent data. Not sure how to re-estimate the rho parameter. But may still be usable today if required. 

Also one needs to consider how to modify the matrix to apply separately to different sectors of the economy, e.g. financials, industrials, commercials, etc. 

In [ ]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from scipy.stats import norm, johnsonsb
from nkutils import read_named_range_to_dataframe as rnr2df
from nktransitionmodule import cum_matrix, unravel, compute_EndCQS, reconstruct_migration_matrix, matpower


In [ ]:
# this matrix is the one resulting from the backsolving process, rounded to 8d.ps and with no further adjustments. 
# while it minimises the differences against the supplied cummulative matrix, its rows do not add up to 1 and the level 
# of accuracy in some instances appears spurious. Also some elements are negative. 

P_nonfin_YE24_EIOPA_original = np.array([
    [0.89939993, 0.09460008, 0.00599997, 0, 0, 0, 0, 0],
    [0.00269941, 0.90170055, 0.08869983, 0.0066, 0.0003, 0, 0, 0],
    [0.00009945, 0.00810052, 0.92649984, 0.0625, 0.0015, 0.0007, 0.0001, 0.0005],
    [0.00000045, 0.00019959, 0.02570012, 0.9361, 0.0325, 0.0035, 0.0006, 0.0014],
    [0.00020021, -0.0000002, 0.00070006, 0.0462, 0.8723, 0.0709, 0.0034, 0.0063],
    [-0.00000028, 0.00010025, 0.00039993, 0.0013, 0.0608, 0.8467, 0.0511, 0.0396],
    [-0.00000018, 0.00000017, 0.00049995, 0.0005, 0.003, 0.182, 0.4832, 0.3308],
    [0.00000006, -0.00000005, 0.00000002, 0, 0, 0, 0, 1],
])
#  we opt to use the matrix below, which was manually produced from the above by ensuring all rows add up to 1
#  and applying judgement to make sure it looks reasonable and does not include spurious accuracy. 

P_nonfin_YE24_EIOPA = np.array([
[0.8994, 0.0946, 0.006, 0, 0, 0, 0, 0],
[0.0027, 0.9017, 0.0886, 0.0066, 0.0003, 0.0001, 0, 0],
[0.0001, 0.0081, 0.9265, 0.0625, 0.0015, 0.0007, 0.0001, 0.0005],
[0, 0.0002, 0.0257, 0.9361, 0.0325, 0.0035, 0.0006, 0.0014],
[0, 0.0002, 0.0007, 0.0462, 0.8723, 0.0709, 0.0034, 0.0063],
[0, 0.0001, 0.0004, 0.0013, 0.0608, 0.8467, 0.0511, 0.0396],
[0, 0, 0.0005, 0.0005, 0.003, 0.182, 0.4832, 0.3308],
[0, 0, 0, 0, 0, 0, 0, 1]
])


#  the below shows the matrix originally used from a paper used to develop a stress mechanism. For now the 
# stress mechanism is used but we opt for using the above EIOPA matrix for our work. 

file_path = 'RiskAndReturn.xlsm'
info_sheet = 'TransitionModule'
TM_range  = 'Input_TM'

TM_df = rnr2df(file_path, info_sheet, TM_range)
TM_array = TM_df.to_numpy()

NumSims = 10000

In [ ]:

rho = 0.0163
sqrt_rho = np.sqrt(rho)
Z = -1

# NEW ARRAY TO USE!!!!
TM_array = P_nonfin_YE24_EIOPA[:-1,:]

CumTM_array = cum_matrix(TM_array)

Next we want to iterate for different values of Z. In [1], the authors considered Z as a function of time, $ Z_t $ and fitted it as an autoregrassive process such that 
$$ Z_t - Z_{t–1} =  – 0.54Z_{t–1} – 1.04 \epsilon_t $$
however for our purposes we will simply sample Z from a standard normal distribution. This can be further investigated as to whether it is reasonable. In particular, sampling from a t distribution with 2 dfs, the equivalent value of 4 in a nomral distribution is 37.5. Such a value of Z would result in most bonds (including AAA) defaulting in most scenarios. At this point this is thought to be too extreme. Ideally one would calibrate the distribution of Z using historic TM data and setting the base_TM accordinlgy. As we lack data, we'll proceed with the above assumption. 

In what comes next, we change the code such that we can sample Z 1000 times and produce 1000 stressed transition matrices. 

Then for each asset we generate a value between 0 and 1. We apply that value through the TM to identify the new rating of the asset. 

For example, let the original rating of an asset be A (so thats a CQS of 2). 
Say the stressed TM for a rating of A in some simulation $S$ is the following: 
0, 0.7%,86.1%,10.4%,1.6%,0.8%,0,0.2%

and say the random value for that bond is 0.90
we have that: 
if the value is between 0 and 0, it upgrades to AAA
if the value is between 0 and 0.7%, it upgrades to AA
if the value is between 0.7% and 86.8%, it remains A
if the value is between 86.8 and 97.2%, it downgrades to BBB
etc... and
if the value is greater than 99.8%, it defaults. 

So in this example, our bond downgrades to BBB. 

and what happens to the spread? 
Say the bond originally had a spread of 100bps.

Under our spread simulation for the same scenario $S$, lets assume that:
Base spreads for A were 120bps and for BBB it was 200bps. 
Spread widening for a BBB has been 150bps. 

In that case, the new spread for our asset (which is now BBB) will be 100 + (200 - 120) + 150 = 330bps.

[Need to think about the new (i.e. stressed) fundamental spread as well - park for a minute.]





OK, now we have all the components, so we are ready to produce the end-CQSs. 
So, first read the start CQS; 
write a routine to produce the stressed cummulative TMs
utilise the routine above to produce the stressed (end) CQS's. 

(need to be careful with sorting the data from excel; risk of comparing apples with pears)

### Update 13/08

We have now updated the P matrix to reflect the YE24 non-financials matrix used for the UK FS tables. While the reconciliation of the FS tables is not exact, it is fairly close. The matrix was reverse-engineered by the YE24 PRA-generated FS data tables. 

In [ ]:
# Load the Excel file
file_path = 'RiskAndReturn.xlsm'
info_sheet = 'Optimiser'
#  assumptions
asset_data_range = 'AssetData_tbl'
# Read the tables into DataFrames
asset_data_df = rnr2df(file_path, info_sheet, asset_data_range)
startCQS_vector = asset_data_df['CQS Base'].values


In [ ]:
Z =  np.random.randn(NumSims) # generator for 1000 stressed TMs
rand_bond_numbers = np.random.rand(NumSims, 100) # bond-specific downgrade numbers

trans_cube = np.zeros((NumSims,7,7))
i=0
for z in Z:
    multiplier = -sqrt_rho * z
    Z_array = norm.ppf(CumTM_array, loc=0, scale=1)
    # Z_shifted_array = norm.ppf(CumTM_array, loc=0, scale=1) + multiplier
    Z_newCumm = norm.cdf(Z_array + multiplier, loc=0, scale=1)
    TM_new = unravel(Z_newCumm)
    CumTM_new = cum_matrix(TM_new)
    trans_cube[i] = CumTM_new
    i+=1
        


In [ ]:
# routine to produce stochastic TMs using power methodology
# chatGPT says that the params below generate fat-tailed rvs between 1 and fat towards 15. and it does!
a,b,loc,scale = 0.5, 0.5, 1.0, 14.0  # tweak a,b if need be
x = johnsonsb.rvs(a, b, loc=loc, scale=scale, size=1000)
rand_bond_numbers = np.random.rand(NumSims, 100) # bond-specific downgrade numbers
trans_cube = np.zeros((NumSims,7,7))
i=0
for z in x:
    TM_new = matpower(P_nonfin_YE24_EIOPA,z)[:-1,:]
    CumTM_new = cum_matrix(TM_new)
    trans_cube[i] = CumTM_new
    i+=1

In [ ]:
import numpy as np
from scipy.stats import johnsonsb

a,b,loc,scale = 0.5, 0.5, 1.0, 14.0

# theoretical 99th percentile
q99_theoretical = johnsonsb.ppf(0.99, a, b, loc=loc, scale=scale)

# empirical with larger sample
x = johnsonsb.rvs(a, b, loc=loc, scale=scale, size=100_000)
q99_empirical = np.percentile(x, 99)

print(q99_theoretical, q99_empirical)

In [ ]:
def extract_P(mymatrix):
    padded = np.hstack([np.ones((mymatrix.shape[0], 1)), mymatrix])
    step1 = - np.diff(padded, axis=1)
    step2 = np.hstack([step1,mymatrix[:,[-1]]])
    step3 = np.vstack([step2,[0,0,0,0,0,0,0,1]])
    return step3

In [ ]:
endCQS_array = compute_EndCQS(trans_cube,rand_bond_numbers,startCQS_vector)

In [ ]:
defaultsPerSim = np.sum(endCQS_array == 999, axis=1)
pd.DataFrame(defaultsPerSim).to_clipboard()

In [ ]:
np.save('Stochastic_TM_10Ksims.npy', endCQS_array)

In [ ]:
pd.DataFrame(startCQS_vector).to_clipboard()

In [ ]:
pd.DataFrame(endCQS_array.T).to_clipboard()

That concludes the code for the CQS / transition section. Next we write the code for the spread widening. 
However, we will need to the endCQS_array for this calculation, but we dont need to worry about how extreme its values become at this point. 

OK, **the rest of this does not appear to be used**; which means that one of the functions in the transitionsim module is not probably used. Park for now. 

In [ ]:
#  NO NEED TO RUN seems to be auxilary/testing block

CumTM_new = cum_matrix(TM_new)
print(np.round(CumTM_new,3))
#  NO NEED TO RUN seems to be auxilary/testing block

np.set_printoptions(suppress=True, precision=10, linewidth=300)
print(np.round(TM_new,8))

In [ ]:
#  NO NEED TO RUN seems to be auxilary/testing block

Z = -9.5
multiplier = -sqrt_rho * Z
Z_array = norm.ppf(CumTM_array, loc=0, scale=1)
# Z_shifted_array = norm.ppf(CumTM_array, loc=0, scale=1) + multiplier
Z_newCumm = norm.cdf(Z_array + multiplier, loc=0, scale=1)
TM_new = unravel(Z_newCumm)
np.set_printoptions(suppress=True, precision=10, linewidth=300)
print(np.round(TM_new,8))

In [ ]:
myarray = matpower(P_nonfin_YE24_EIOPA,6)
print(myarray)

In [ ]:

tempEndCQS = endCQS_array.copy()
tempEndCQS[tempEndCQS==999] = 7
CQS_change_array = tempEndCQS - startCQS_vector


In [ ]:
print(np.sum(endCQS_array==999))

In [ ]:
print(np.sum(CQS_change_array,axis=1))
print(np.sum(CQS_change_array * (CQS_change_array > 0), axis=1))
print(np.sum(CQS_change_array * (CQS_change_array < 0), axis=1))

In [ ]:
print(np.max(endCQS_array, axis=0))

# Backsolve P from PD probabilities
The code below attempts to backsolve matrix P from the matrix K (7 x 30), where K is the default column of the P^n matrix, n=1,2,...30.
(dont think its working and has not been tested; there is a better way saved elsewhere by Neil Sheridan).

In [ ]:
PD_prob_Nfins_base = np.array([
    [0, 0.000003, 0.000014672608, 0.000041102416508, 0.000088817322877, 0.000164816791747, 0.00027658594293, 0.000432088886205, 0.000639738906874, 0.000908346259768, 0.001247046406297, 0.001665212700111, 0.002172358012172, 0.002778029789704, 0.003491702732482, 0.004322672772323, 0.005279955450947, 0.006372191173312, 0.007607559212078, 0.008993701781895, 0.010537659005653, 0.01224581516616, 0.014123856277526, 0.016176738718631, 0.018408668441589, 0.020823090095054, 0.023422685278695, 0.026209379064285, 0.02918435387382, 0.032348069789728],
    [0, 0.00005548, 0.000175755432, 0.000370816299506, 0.00065135651006, 0.001028719365639, 0.001514757357821, 0.002121630671247, 0.002861572879714, 0.0037466482817, 0.004788519137421, 0.005998235103385, 0.007386052207324, 0.008961284905213, 0.010732192001819, 0.012705895311828, 0.014888328702055, 0.017284214422081, 0.019897063265927, 0.022729195004883, 0.025781775609235, 0.029054867972723, 0.032547493122016, 0.036257699200754, 0.040182635839907, 0.044318631846468, 0.048661274449687, 0.053205488631029, 0.05794561532658, 0.062875487526816],
    [0.0005, 0.001121, 0.001879901501, 0.002795097922673, 0.003886027616714, 0.005172144740545, 0.006671907180534, 0.008401983520253, 0.010376715417387, 0.012607808802212, 0.015104208647869, 0.017872111485316, 0.020915075790624, 0.024234197931575, 0.027828328559385, 0.031694310503457, 0.035827224252719, 0.040220631078744, 0.044866806932462, 0.049756962592493, 0.054881447306963, 0.060229934476956, 0.065791588880219, 0.071555215610171, 0.077509391372244, 0.083642579088066, 0.089943226947606, 0.096399853151088, 0.103001117620048, 0.109735881948771],
    [0.0014, 0.00326522, 0.005613847514, 0.008475931512039, 0.011872909001769, 0.015812076617498, 0.020287034678944, 0.025280175383975, 0.03076556239078, 0.036711562169332, 0.043083027656152, 0.049843017032181, 0.056954098428532, 0.064379310112824, 0.072082844425008, 0.080030515298282, 0.088190059114381, 0.096531309094122, 0.105026275154538, 0.113649154301996, 0.122376291067064, 0.131186103036839, 0.140058983012296, 0.148977186538322, 0.157924711375052, 0.166887173778985, 0.175851685142223, 0.18480673151853, 0.193742057781945, 0.202648557566942],
    [0.0063, 0.01579288, 0.028331178996, 0.043380475167901, 0.060331815774793, 0.078623854413477, 0.097781946197786, 0.117422558924873, 0.1372445057063, 0.157016319458537, 0.176563607594322, 0.195757797171189, 0.214506636597798, 0.232746396937469, 0.250435566624356, 0.267549800899521, 0.284077901162609, 0.300018628876986, 0.315378190649784, 0.330168260576392, 0.344404431303116, 0.358105006379939, 0.371290063752364, 0.383980734239754, 0.396198650127247, 0.407965528058084, 0.419302857675568, 0.430231673278169, 0.440772390401942, 0.450944692961761],
    [0.0396, 0.09041826, 0.142563183651, 0.192079329268955, 0.237633114757034, 0.279011104032476, 0.316446316877944, 0.35032152256156, 0.381043136785914, 0.40899148811493, 0.434504586714443, 0.457876024530377, 0.47935835860506, 0.499168202235578, 0.517491446907173, 0.534488016289845, 0.550295978168778, 0.565035015447931, 0.578809322075632, 0.59171000680118, 0.603817085340573, 0.61520113228684, 0.625924653281009, 0.636043227698044, 0.645606463125671, 0.654658795354874, 0.663240161357221, 0.671386567619271, 0.679130572052456, 0.686501694328697],
    [0.3308, 0.49786961, 0.587876290622, 0.640897063464528, 0.675575675933231, 0.700676268412351, 0.720393157504678, 0.736794028687569, 0.750946500524264, 0.763439504604384, 0.774626128105617, 0.784737964323598, 0.793939951781463, 0.802357460463827, 0.810090283647831, 0.817220330682103, 0.823816194565943, 0.829936095586153, 0.83562992605547, 0.840940758000761, 0.845906003343959, 0.850558332510881, 0.854926415383359, 0.859035526287586, 0.862908042146999, 0.866563855256058, 0.870020717074922, 0.873294525886688, 0.876399568527324, 0.879348724384857]
])
print(PD_prob_Nfins_base.shape)
K = PD_prob_Nfins_base.transpose()
M = reconstruct_migration_matrix(K)
print(matpower(M,5))